In [1]:
#%matplotlib notebook
%matplotlib inline 

%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd


sys.path.append("../../pycode/")
#from magus_util import read_nvprof_trace, parse_nvprof_trace, getruntime, sort_dict_by_val, genNNFeat, testFeat
from magus_util import *
from magus_contention import *

# load dedicated execution time

In [2]:
dedicated_app_dd = np.load("../../09_case_study/dedicated_app_runtime.npy").item()

In [3]:
# test 
dedicated_app_dd['rodinia_b+tree']

2326.0

# load the app2metric 

In [4]:
app2metric_dd = np.load("../../07_sim_devid/similarity/app2metric_dd.npy").item()

In [5]:
# test
# app2metric_dd['rodinia_b+tree']
# app2metric_dd['rodinia_btree']

# similarity distance

In [6]:
app2app_sim_dd = np.load('app2app_sim_dd.npy').item()

In [7]:
# test
# app2app_sim_dd['rodinia_b+tree']
# app2app_sim_dd['rodinia_btree']

#  features from traces

In [8]:
app_iobound_dd = np.load('app_iobound_dd.npy').item()

app2newfeat_dd = np.load('app2newfeat_dd.npy').item()

# ground truth: speedup for run2

In [9]:
#
# /magus/09_case_study/testall_greedy/2-groundtruth_cases_from_run2log.ipynb
# 
run2_groundTruth_dd = np.load("../../09_case_study/testall_greedy/app_run2_speedup_dd.npy").item()

In [10]:
run2_groundTruth_dd['rodinia_b+tree']

{'cudasdk_BlackScholes': [0.8335931592320622, 0.7816119826962799, 1],
 'cudasdk_FDTD3d': [0.9518470169867453, 0.9344981592524537, 1],
 'cudasdk_MCEstimatePiInlineP': [0.834589163975601, 0.587002358490566, 0],
 'cudasdk_MCEstimatePiInlineQ': [0.8413300683264187, 0.67340857605178, 0],
 'cudasdk_MCEstimatePiP': [0.833491598791688, 0.6563596149421449, 0],
 'cudasdk_MCEstimatePiQ': [0.8110181311018131, 0.6733909822700007, 0],
 'cudasdk_MCSingleAsianOptionP': [0.8571418042724428, 0.6422829581993569, 0],
 'cudasdk_SobolQRNG': [0.8572492103798653, 0.6921043916486681, 0],
 'cudasdk_batchCUBLAS': [0.8170003512469266, 0.6819389396731861, 0],
 'cudasdk_binomialOptions': [0.9305228290134298, 0.9473272151632849, 1],
 'cudasdk_boxFilterNPP': [0.8329961000311568, 0.5878959868548508, 0],
 'cudasdk_c++11Cuda': [0.8332945226827355, 0.639772890909465, 0],
 'cudasdk_concurrentKernels': [0.7989475600498741, 0.2965081723625557, 0],
 'cudasdk_convolutionFFT2D': [1.0, 0.9719302434276951, 1],
 'cudasdk_convolut

# study all the application

In [11]:
def find_large_and_small_jobs(target_app, app2newfeat_dd):
    target_appdur  = app2newfeat_dd[target_app][0]
    target_prepare = app2newfeat_dd[target_app][1]
    
    large_job_list = []
    small_job_list = []
    total_job_list = []

    for key, value in app2newfeat_dd.iteritems():
        if key <> target_app:
            total_job_list.append(key)
            
            my_appdur  = value[0]
            my_prepare = value[1]
            #print my_appdur, my_prepare
            if my_prepare >= target_appdur and my_appdur > target_appdur * 2.0:
                large_job_list.append(key)

            if my_appdur <= target_prepare and my_appdur * 2.0 < target_appdur:
                small_job_list.append(key)
        
    mylist = []
    
    if large_job_list:  # prefer large job to corun
        mylist = large_job_list
    else:
        if small_job_list:   # if no large job, use the small job list
            mylist = small_job_list
        else:
            mylist = total_job_list  # if there is no large/small job, use the orginial set
    
    
    #print("large_job_list = {} \t small_job_list = {}".format(len(large_job_list), len(small_job_list)))
    
    return mylist
    

In [12]:
def select_opposite_type_app(target_app, job_list, app_iobound_dd):
    app1_type = app_iobound_dd[target_app]
    opp_type_list = []
    for app2 in job_list:
        app2_type = app_iobound_dd[app2]
        if app1_type <> app2_type:
            opp_type_list.append(app2)
    return opp_type_list
            

In [13]:
def find_corun_list(app1, app2newfeat_dd, job_list, DEV_THREAD_MAX):
    #[a1_thd_max, _, a1_reg_max, _, a1_sm_max, _, a1_trans_max, _] =app2newfeat_dd[app1][3:]
    a1_thd_max = app2newfeat_dd[app1][3]
    #print a1_thd_max, a1_reg_max, a1_sm_max, a1_trans_max, "\n"

    avail_thd = DEV_THREAD_MAX - a1_thd_max

    corun_list = []
    if avail_thd > 0:  # for cases, where there are more threads can be launched
        for i in job_list:
            #[threads_max, _, reg_max, _, sm_max, _, trans_max, _] =app2newfeat_dd[i][3:]
            threads_max = app2newfeat_dd[i][3]
            
            #print threads_max, reg_max, sm_max, trans_max
            if threads_max <= avail_thd:
                corun_list.append(i)
    return corun_list

In [14]:
def find_least_sim(app1, app2app_sim_dd, job_list):
    app1_sim_dd = app2app_sim_dd[app1]

    dist_dd = {}
    for app2 in job_list:
        dist_dd[app2] = app1_sim_dd[app2]

    sorted_dist = sorted(dist_dd.items(), key=operator.itemgetter(1))
    #print sorted_dist
    #print sorted_dist[-1]

    # the least similar one is the last, since sorting in non-decreasing order
    return sorted_dist[-1]

In [15]:
# # example

# TARGET_APP = "cudasdk_concurrentKernels"
# large_job_list, small_job_list = select_large_and_small_jobs(TARGET_APP, app2newfeat_dd)
# print len(large_job_list), len(small_job_list)


# opp_type_list = select_opposite_type_app(TARGET_APP, large_job_list, app_iobound_dd)
# print opp_type_list


In [16]:
# for i, _ in app_iobound_dd.iteritems():
#     print i
#     print app2newfeat_dd[i], "\n"

In [17]:
count = 0
DEV_THREAD_MAX = 28 * 2048

for app1, app1_iobound in app_iobound_dd.iteritems():
    print("{} \t iobound = {}".format(app1, app1_iobound))
    
    TARGET = None
    CNT = True
    
    #===================================#
    # step1 : separate the large and small job list
    #===================================#
    #print "Start step1:"
    job_list = find_large_and_small_jobs(app1, app2newfeat_dd)  
    #print len(job_list)
    
    KeepGo = True
    if len(job_list) == 1:
        KeepGo = False
        TARGET = job_list[0]
        print("step 1 => Found: {}".format(TARGET))
        
    CNT = (CNT and KeepGo)
    
    #===================================#
    # step2 : prefer opposite type job to corun
    #===================================#
    
    #print job_list
    
    if CNT:
        #print "Start step2\n"
        opp_type_list = select_opposite_type_app(app1, job_list, app_iobound_dd)
        
       
        if opp_type_list:
            job_list = opp_type_list   # update job_list if there are opposite types
        
        KeepGo = True
        if len(job_list) == 1:
            KeepGo = False
            TARGET = job_list[0]
            print("step 2 => Found: {}".format(TARGET))
            
        CNT = (CNT and KeepGo)
        

    #===================================#
    # step3 : check the thread limitation
    #===================================#
    if CNT:
        #print "Start step 3\n"
        corun_list = find_corun_list(app1, app2newfeat_dd, job_list, DEV_THREAD_MAX)
        if corun_list:
            job_list = corun_list
        
        KeepGo = True
        if len(job_list) == 1:
            KeepGo = False
            TARGET = job_list[0]
            print("step 3 => Found: {}".format(TARGET))
        
        CNT = (CNT and KeepGo)
        
    #===================================#
    # step4 : similarity analysis
    #===================================#
    if CNT:
        #print "Start step 4\n"
        least_sim = find_least_sim(app1, app2app_sim_dd, job_list) # return value is a tuple (appName, distVal)
        TARGET = least_sim[0]
        print("step 4 => Found: {}".format(TARGET))
        
        
        
#     if TARGET == "rodinia_b+tree":
#         TARGET = "rodinia_btree"
#     if TARGET == "cudasdk_c++11Cuda":
#         TARGET = "cudasdk_c11Cuda"
        
        
#     if app1 == "rodinia_b+tree":
#         app1 = "rodinia_btree"
#     if app1 == "cudasdk_c++11Cuda":
#         app1 = "cudasdk_c11Cuda"
        
        
    print run2_groundTruth_dd[app1][TARGET]
    
        
    #
    # baseline: using similarity only
    # 
    dist_dd = app2app_sim_dd[app1]
    sorted_dist = sorted(dist_dd.items(), key=operator.itemgetter(1))
    least_sim_app = sorted_dist[-1][0]
    print("Using just similarity => Found: {}".format(least_sim_app)) 
    print run2_groundTruth_dd[app1][least_sim_app]
    
    print("\n")
        
    count += 1
    
    #if count == 10: break
    

rodinia_b+tree 	 iobound = 1
step 1 => Found: cudasdk_FDTD3d
[0.9518470169867453, 0.9344981592524537, 1]
Using just similarity => Found: cudasdk_segmentationTreeThrust
[0.9631469979296067, 0.7580353084082898, 1]


rodinia_heartwall 	 iobound = 0
step 3 => Found: lonestar_dmr
[0.6974264705882354, 0.39900160892679076, 0]
Using just similarity => Found: cudasdk_segmentationTreeThrust
[0.6737582355134877, 0.7223867453537011, 0]


cudasdk_convolutionFFT2D 	 iobound = 1
step 1 => Found: cudasdk_FDTD3d
[1.0247308364283323, 0.9353177174346993, 1]
Using just similarity => Found: cudasdk_interval
[0.8776904805408369, 0.7392399260007814, 1]


shoc_lev1BFS 	 iobound = 0
step 4 => Found: cudasdk_radixSortThrust
[1.1643995586589029, 0.7856657082640344, 1]
Using just similarity => Found: poly_3mm
[0.9662307111399223, 0.5953285420944558, 0]


rodinia_lavaMD 	 iobound = 0
step 4 => Found: cudasdk_convolutionFFT2D
[0.5864336348081154, 0.9309302325581396, 0]
Using just similarity => Found: poly_3mm
[0.58

# merge the selection into a function

In [ ]:
# def app_select_01(app1, app_iobound_dd, ):

    
#     app1_iobound = app_iobound_dd[app1]
#     print("{} \t iobound = {}".format(app1, app1_iobound))
    
#     count = 0
#     DEV_THREAD_MAX = 28 * 2048
#     TARGET = None
#     CNT = True

    
    
#         #===================================#
#         # step1 : separate the large and small job list
#         #===================================#
#         #print "Start step1:"
#         job_list = find_large_and_small_jobs(app1, app2newfeat_dd)  
#         #print len(job_list)

#         KeepGo = True
#         if len(job_list) == 1:
#             KeepGo = False
#             TARGET = job_list[0]
#             print("step 1 => Found: {}".format(TARGET))

#         CNT = (CNT and KeepGo)

#         #===================================#
#         # step2 : prefer opposite type job to corun
#         #===================================#

#         #print job_list

#         if CNT:
#             #print "Start step2\n"
#             opp_type_list = select_opposite_type_app(app1, job_list, app_iobound_dd)


#             if opp_type_list:
#                 job_list = opp_type_list   # update job_list if there are opposite types

#             KeepGo = True
#             if len(job_list) == 1:
#                 KeepGo = False
#                 TARGET = job_list[0]
#                 print("step 2 => Found: {}".format(TARGET))

#             CNT = (CNT and KeepGo)


#         #===================================#
#         # step3 : check the thread limitation
#         #===================================#
#         if CNT:
#             #print "Start step 3\n"
#             corun_list = find_corun_list(app1, app2newfeat_dd, job_list, DEV_THREAD_MAX)
#             if corun_list:
#                 job_list = corun_list

#             KeepGo = True
#             if len(job_list) == 1:
#                 KeepGo = False
#                 TARGET = job_list[0]
#                 print("step 3 => Found: {}".format(TARGET))

#             CNT = (CNT and KeepGo)

#         #===================================#
#         # step4 : similarity analysis
#         #===================================#
#         if CNT:
#             #print "Start step 4\n"
#             least_sim = find_least_sim(app1, app2app_sim_dd, job_list) # return value is a tuple (appName, distVal)
#             TARGET = least_sim[0]
#             print("step 4 => Found: {}".format(TARGET))



#     #     if TARGET == "rodinia_b+tree":
#     #         TARGET = "rodinia_btree"
#     #     if TARGET == "cudasdk_c++11Cuda":
#     #         TARGET = "cudasdk_c11Cuda"


#     #     if app1 == "rodinia_b+tree":
#     #         app1 = "rodinia_btree"
#     #     if app1 == "cudasdk_c++11Cuda":
#     #         app1 = "cudasdk_c11Cuda"


#         print run2_groundTruth_dd[app1][TARGET]


#         #
#         # baseline: using similarity only
#         # 
#         dist_dd = app2app_sim_dd[app1]
#         sorted_dist = sorted(dist_dd.items(), key=operator.itemgetter(1))
#         least_sim_app = sorted_dist[-1][0]
#         print("Using just similarity => Found: {}".format(least_sim_app)) 
#         print run2_groundTruth_dd[app1][least_sim_app]

#         print("\n")

#         count += 1
    
#         #if count == 10: break
    